In [1]:
from neural_network.binary_nn import NN
from datasets.sipm_dataset import SiPMDataset
from genetic_algorithm.algorithm import GeneticAlgorithm

In [2]:
# GA parameters
mutation_prob = 0.05
pop_size = 10       # small population for demo
ngen = 5            # few generations
n_samples = 128

# nn = NN((genome_length, 64, 128, 2), input=SiPMDataset(n_samples=genome_length))
nn = NN((n_samples, 16, 128, 2), input=SiPMDataset(n_samples=n_samples))

ga = GeneticAlgorithm(
    nn=nn, 
    pop_size=pop_size,
    ngen=ngen,
    elite_size=2
)

In [3]:
pop, log, hof = ga.run()

Create init population...
Start evolution...
   	      	        divers        	                          fitness                           	         size         
   	      	----------------------	------------------------------------------------------------	----------------------
gen	nevals	gen	max	nevals	avg	diversity	gen	max	min	nevals	time	gen	max  	nevals
0  	10    	0  	0  	10    	0  	0.1      	0  	0  	0  	10    	9 s 	0  	0.108	10    
1  	0     	1  	0  	0     	0  	0.1      	1  	0  	0  	0     	18 s	1  	0.108	0     
2  	0     	2  	0  	0     	0  	0.1      	2  	0  	0  	0     	27 s	2  	0.108	0     
3  	0     	3  	0  	0     	0  	0.1      	3  	0  	0  	0     	34 s	3  	0.105	0     
4  	0     	4  	0  	0     	0  	0.1      	4  	0  	0  	0     	43 s	4  	0.109	0     
5  	0     	5  	0  	0     	0  	0.1      	5  	0  	0  	0     	52 s	5  	0.109	0     
Evolution finished.


In [4]:
print("Best individual:", hof[0])
print("Best fitness:", ga.nn.fitness(hof[0]))
print("GA run complete.\n")

Best individual: [np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int6

# Testing

In [5]:
nn = NN((2, 8, 2), input=SiPMDataset())
x, y = SiPMDataset().load_data()
print(f"Inputs (X): \n{x}\n\n\nLabels (y): \n{y}\n\n\n")

Inputs (X): 
[[ 8  8 20 ...  8  8  8]
 [ 8  8 15 ...  8  8  8]
 [ 8  8 22 ...  8  8  8]
 ...
 [ 8  8 13 ...  8  8  8]
 [ 8  8 34 ...  8  8  8]
 [ 8  8 11 ...  8  8  8]]


Labels (y): 
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]



